lets test the different QA BERT models on each of the financial,scientific and Biomedical domain.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")
!ls

 Complete.pdf				  PDF_Folder_Financial
'covid_question_answer_dataset (1).csv'   PDF_Folder_Scientific
 covid_question_answer_dataset.csv	 ' QA_system_for_BioMedical_data.ipynb'
 CrossDomainAnalysis.ipynb		  QA_system_for_scientific_data.ipynb
 DistilBERT.ipynb			  question_answer_dataset_financial.csv
 Make_dataset_from_PDFs.ipynb		  scientific_question_answer_dataset.csv
 PDF_Folder_Biomedical


Test Biomedical and Scientific PDFs on BERT Large model

In [ ]:
import pandas as pd
df_BioMedical = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/covid_question_answer_dataset.csv")
df_Scientific = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/scientific_question_answer_dataset.csv")

In [ ]:
df_BioMedical

,Question,Context,GroundTruth
0,What vulnerabilities has the COVID-19 pandemic...,"\nIntroduction\nThe COVID-19 pandemic, a publi...",The COVID-19 pandemic has revealed vulnerabili...
1,How has the COVID-19 pandemic affected the bio...,"\nIntroduction\nThe COVID-19 pandemic, a publi...",The COVID-19 pandemic has affected the biomedi...
2,What challenges and lessons learned are discus...,"\nIntroduction\nThe COVID-19 pandemic, a publi...",The paper discusses the current status of rese...
3,How did the COVID-19 pandemic highlight the ne...,"\nIntroduction\nThe COVID-19 pandemic, a publi...",The COVID-19 pandemic highlighted the necessit...
4,What is the current status of research describ...,"\nIntroduction\nThe COVID-19 pandemic, a publi...",The paper describes the current status of rese...
5,How did public-private collaborations contribu...,"\nIntroduction\nThe COVID-19 pandemic, a publi...",Public-private collaborations during the COVID...
6,What strategies did the National Institutes of...,"\nIntroduction\nThe COVID-19 pandemic, a publi...",The NIH adopted strategies such as investing i...
7,What priorities are outlined in The Science Ag...,"\nIntroduction\nThe COVID-19 pandemic, a publi...",The Science Agenda for COVID-19 by the CDC out...
8,"How did research funders, including non-profit...","\nIntroduction\nThe COVID-19 pandemic, a publi...","Research funders, including non-profit entitie..."
9,What efforts did the Patient-Centered Outcomes...,"\nIntroduction\nThe COVID-19 pandemic, a publi...",PCORI undertook efforts focused on adaptations...


In [ ]:
import torch
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from sklearn.metrics import f1_score
import re
def answer_question4(question, answer_text, ground_truth):
    ground_truth_processed = re.sub(r' ##', '', ground_truth)
    # Tokenize and preprocess the ground truth
    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
    print('The input has a total of {:} tokens.'.format(len(input_ids)))
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    # Construct the list of 0s and 1s.
    segment_ids = [0] * num_seg_a + [1] * num_seg_b
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)
    outputs = model(input_ids=torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    predicted_answer = tokens[start_index]
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(start_index + 1, end_index + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            predicted_answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            predicted_answer += ' ' + tokens[i]

    # Tokenize and preprocess the predicted answer
    predicted_answer_processed = re.sub(r' ##', '', predicted_answer)
    # Ensure that ground truth and predicted answer have the same length
    max_length = max(len(ground_truth_processed), len(predicted_answer_processed))
    ground_truth_processed = ground_truth_processed.ljust(max_length)
    predicted_answer_processed = predicted_answer_processed.ljust(max_length)

    # Calculate F1 score for ground truth and predicted answer
    f1_ground_truth_predicted = f1_score(list(ground_truth_processed), list(predicted_answer_processed), average='weighted', zero_division=1)
    return predicted_answer, f1_ground_truth_predicted

In [ ]:
# Loop through rows and call the function
for index, row in df_BioMedical.iterrows():
    user_question = row['Question']
    context = row['Context']
    ground_truth = row['GroundTruth']
    # Call answer_question function
    ans, f1_value = answer_question4(user_question,context,ground_truth)
    # Print the result
    print(f"Question: {user_question}")
    print(f"Context: {context}")
    print(f"Answer: {ans}")
    print(f"F1 Score: {f1_value}")
    print("=" * 50)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


The input has a total of 512 tokens.


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Question: What vulnerabilities has the COVID-19 pandemic revealed in the health care system and public health infrastructure?
Context: 
Introduction
The COVID-19 pandemic, a public health emergency of unprecedented scale and consequences, has revealed vulnerabilities in our health care system and public health infrastructure, yet also serves as a remarkable learning opportunity for transformational changes. Effects of the COVID-19 pandemic touch every aspect of life in ways not previously imagined—the biomedical and health research enterprises are no exception. Preexisting stresses in the research sector’s workforce, processes, and organizations have been exacerbated in the sector’s quest to effectively generate meaningful information in response to the pandemic and deliver research in new and innovative ways. The COVID-19 pandemic revealed the necessity to enhance the ability for researchers to share data through interoperable and customizable systems to enable rigor, reproducibility,

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

Question: What strategies did the National Institutes of Health (NIH) adopt to accelerate the development of therapeutic interventions, vaccines, and diagnostics?
Context: 
Introduction
The COVID-19 pandemic, a public health emergency of unprecedented scale and consequences, has revealed vulnerabilities in our health care system and public health infrastructure, yet also serves as a remarkable learning opportunity for transformational changes. Effects of the COVID-19 pandemic touch every aspect of life in ways not previously imagined—the biomedical and health research enterprises are no exception. Preexisting stresses in the research sector’s workforce, processes, and organizations have been exacerbated in the sector’s quest to effectively generate meaningful information in response to the pandemic and deliver research in new and innovative ways. The COVID-19 pandemic revealed the necessity to enhance the ability for researchers to share data through interoperable and customizable syst

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Question: What priorities are outlined in The Science Agenda for COVID-19 established by the Centers for Disease Control and Prevention (CDC)?
Context: 
Introduction
The COVID-19 pandemic, a public health emergency of unprecedented scale and consequences, has revealed vulnerabilities in our health care system and public health infrastructure, yet also serves as a remarkable learning opportunity for transformational changes. Effects of the COVID-19 pandemic touch every aspect of life in ways not previously imagined—the biomedical and health research enterprises are no exception. Preexisting stresses in the research sector’s workforce, processes, and organizations have been exacerbated in the sector’s quest to effectively generate meaningful information in response to the pandemic and deliver research in new and innovative ways. The COVID-19 pandemic revealed the necessity to enhance the ability for researchers to share data through interoperable and customizable systems to enable rigor,

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Question: How did research funders, including non-profit entities, contribute to the pandemic response?
Context: 
Introduction
The COVID-19 pandemic, a public health emergency of unprecedented scale and consequences, has revealed vulnerabilities in our health care system and public health infrastructure, yet also serves as a remarkable learning opportunity for transformational changes. Effects of the COVID-19 pandemic touch every aspect of life in ways not previously imagined—the biomedical and health research enterprises are no exception. Preexisting stresses in the research sector’s workforce, processes, and organizations have been exacerbated in the sector’s quest to effectively generate meaningful information in response to the pandemic and deliver research in new and innovative ways. The COVID-19 pandemic revealed the necessity to enhance the ability for researchers to share data through interoperable and customizable systems to enable rigor, reproducibility, and efficiency. This 

In [ ]:
from sklearn.metrics import f1_score
import re

def answer_question4(question, answer_text, ground_truth):
    ground_truth_processed = re.sub(r' ##', '', ground_truth)

    # Tokenize and preprocess the ground truth
    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
    print('The input has a total of {:} tokens.'.format(len(input_ids)))
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0] * num_seg_a + [1] * num_seg_b
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    outputs = model(input_ids=torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    predicted_answer = tokens[start_index]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(start_index + 1, end_index + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            predicted_answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            predicted_answer += ' ' + tokens[i]

    # Tokenize and preprocess the predicted answer
    predicted_answer_processed = re.sub(r' ##', '', predicted_answer)

    # Ensure that ground truth and predicted answer have the same length
    max_length = max(len(ground_truth_processed), len(predicted_answer_processed))
    ground_truth_processed = ground_truth_processed.ljust(max_length)
    predicted_answer_processed = predicted_answer_processed.ljust(max_length)

    # Calculate F1 score for ground truth and predicted answer
    f1_ground_truth_predicted = f1_score(list(ground_truth_processed), list(predicted_answer_processed), average='weighted', zero_division=1)
    return predicted_answer, f1_ground_truth_predicted

In [ ]:
for index, row in df_Scientific.iterrows():
    user_question = row['Question']
    context = row['Context']
    ground_truth = row['GroundTruth']
    # Call your answer_question function
    ans, f1 = answer_question4(user_question,context,ground_truth)

    # Print the result
    print(f"Question: {user_question}")
    print(f"Context: {context}")
    print(f"Answer: {ans}")
    print(f"F1 Score: {f1}")
    print("=" * 50)


The input has a total of 134 tokens.
Question: What does this paper present?
Context: 
This paper presents an overview of a quality scoring system that
utilizes pre-trained deep neural network models. Two types of DL
models, a classification and extractive question answering (EQA)
models are used to implement components of the system.
.
.
.
 The number of animals can be an important factor in an in-vivo study; however, the same criteria are not applicable for an in-vitro study. Copyright is held by the authors.
APPLIED COMPUTING REVIEW MAR. 2022, VOL. 22, NO. 1 31
Figure 1: Overview of a paper quality scoring system
Figure 1 shows an overview of the software system. T

Answer: an overview of a quality scoring system
F1 Score: 0.026548672566371678
The input has a total of 141 tokens.
Question: What types of DL models are used in the quality scoring system?
Context: 
This paper presents an overview of a quality scoring system that
utilizes pre-trained deep neural network models. Two type

####